In [1]:
# %run ./connect_to_api.ipynb
from collections import Counter
from collections import defaultdict
import requests
import pandas as pd

# map_id = input('Please enter a map ID: ')

In [2]:
def convert_clu_no_to_group_name (clu_no):
    """
    This function takes cluster numbers are returns the name of the group it belongs to
    Inputs:
        clu_no: Given cluster number
    Outputs:
        group_name: Associated group name
    """
    group_no = cluster_num_to_group_num[str(clu_no)]
    group_name = group_no_to_group_name[group_no]
    return (group_name)


def get_group_density ():
    """
    This function runs the entirety of the Group Density script
    Outputs:
        Prints the calculated densities for each group in a given map
    """
    map_id = input('>> Please enter a map ID: ')
    #get all the date
    r = requests.get("https://api.graphika.com/clustermaps/{0}/links".format(map_id),auth=(username,pswd))
    link_hash = r.json()
    r = requests.get("https://api.graphika.com/clustermaps/{0}/nodes".format(map_id),auth=(username,pswd))
    node_hash = r.json()
    r = requests.get("https://api.graphika.com/clustermaps/{0}/clusters".format(map_id),auth=(username,pswd))
    cluster_hash = r.json()
    r = requests.get("https://api.graphika.com/clustermaps/{0}/groups".format(map_id),auth=(username,pswd))
    group_hash = r.json()
    
    
    #converting to node by group name
    nodes_by_clu_no = dict([(n["screenname"],n["attentive_cluster_id"]) for n in node_hash])
    cluster_num_to_group_num = dict([(n["id"],n["group"]) for n in cluster_hash["clusters"]])
    nodes_by_group_num = dict([(k,cluster_num_to_group_num[str(v)]) for k,v in nodes_by_clu_no.items()])
    group_num_to_group_name = dict([(k,v["name"]) for k,v in group_hash.items()])

    node_group_df = pd.DataFrame(list(nodes_by_group_num.items()), columns=["account_name","group_num"])
    node_group_df["group_name"] = node_group_df['group_num'].map(group_num_to_group_name)
    node_group_df.set_index("account_name", inplace=True)
    
    #get group node counts
    group_counts = node_group_df["group_name"].value_counts()
    group_counts_df = pd.DataFrame(group_counts)
    
    
    #get group internal connections
    d = defaultdict(int)

    for n,v in link_hash.items():
        node = node_group_df.loc[n,"group_num"]
        arcs = [node_group_df.loc[x,"group_num"] for x in v]
        count_arcs = Counter(arcs)
        if node in count_arcs:
            internal_conn = count_arcs[node]
            d[node] += internal_conn       
            
    new_dict = {}

    for k,v in d.items():
        group_name = group_num_to_group_name[k]
        new_dict[group_name] = v
           
    group_arcs = pd.DataFrame.from_dict(new_dict, orient="index")
    
    
    
    #combine and calculate density
    group_density = group_counts_df.join(group_arcs)
    group_density.columns = ["nodes","arcs"]
    group_density["density"] = group_density.arcs * 1.0 / ((group_density.nodes * group_density.nodes) -1)
    group_density.density = group_density.density*100
    
    display(group_density)
    if input('>> Would you like to save these results to a CSV? (y/n) \n') == 'y':
        print_csv(group_density)
    return